In [2]:
pip install torch

   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/109.3 MB 2.0 MB/s eta 0:00:56
   ---------------------------------------- 0.2/109.3 MB 2.9 MB/s eta 0:00:38
   ---------------------------------------- 0.4/109.3 MB 3.1 MB/s eta 0:00:35
   ---------------------------------------- 0.5/109.3 MB 2.9 MB/s eta 0:00:38
   ---------------------------------------- 0.6/109.3 MB 2.7 MB/s eta 0:00:41
   ---------------------------------------- 0.7/109.3 MB 2.7 MB/s eta 0:00:41
   ---------------------------------------- 0.8/109.3 MB 2.6 MB/s eta 0:00:43
   ---------------------------------------- 0.9/109.3 MB 2.6 MB/s eta 0:00:42
   ---------------------------------------- 1.1/109.3 MB 2.6 MB/s eta 0:00:43
   ---------------------------------------- 1.1/109.3 MB 2.5 MB/s eta 0:00:44
   ---------------------------------------- 1.2/109.3 MB 2.5 MB/s eta 0:00:44
   ---------------------------------------- 1.3/109.3 MB 2.4 MB/s eta 0

ERROR: Could not install packages due to an OSError: [WinError 5] Zugriff verweigert: 'c:\\Python312\\share'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn

In [18]:
player_stats = pd.read_csv('strict_player_stats.csv')
fifa_data = pd.read_csv('strict_fifa_data.csv')
validation_df = pd.read_csv('strict_validation.csv')

merged_data = pd.merge(
    player_stats, 
    validation_df[['Stats_Player', 'FIFA_Overall']], 
    left_on='Player', 
    right_on='Stats_Player',
    how='inner'
)

In [11]:
def prepare_features_for_training(df, exclude_cols=None):
    if exclude_cols is None:
        exclude_cols = ['Rk', 'Born', 'MP', 'Starts', 'Min', '90s', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age']
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    feature_cols = [col for col in numeric_cols if col not in exclude_cols and df[col].nunique() > 1]
    
    features_df = df[feature_cols].copy()
    features_df = features_df.fillna(features_df.median())
    
    for col in features_df.columns:
        Q1 = features_df[col].quantile(0.25)
        Q3 = features_df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        features_df[col] = np.clip(features_df[col], lower_bound, upper_bound)
    
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(features_df)
    
    return X_scaled, scaler, features_df.columns.tolist(), features_df

X_scaled, feature_scaler, feature_names, features_df = prepare_features_for_training(merged_data)
y_target = merged_data['FIFA_Overall']

print(f"🎯 Features: {len(feature_names)}")
print(f"🔢 Target Range: {y_target.min()} - {y_target.max()}")

🎯 Features: 132
🔢 Target Range: 47 - 91


In [12]:
def find_optimal_clusters(X, max_k=10):
    inertias = []
    for k in range(2, max_k + 1):
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X)
        inertias.append(kmeans.inertia_)
    return inertias

inertias = find_optimal_clusters(X_scaled)
optimal_k = 6  

kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)
merged_data['Cluster'] = cluster_labels

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
merged_data['PC1'] = X_pca[:, 0]
merged_data['PC2'] = X_pca[:, 1]

print(f"Clustering abgeschlossen: {optimal_k} Cluster")

Clustering abgeschlossen: 6 Cluster


In [13]:
class PlayerRatingPredictor(nn.Module):
    def __init__(self, input_size):
        super(PlayerRatingPredictor, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid() 
        )
    
    def forward(self, x):
        output = self.network(x)
        return output * 100  

def train_pytorch_model(X, y):
    y_scaled = y / 100.0
    
    X_tensor = torch.FloatTensor(X)
    y_tensor = torch.FloatTensor(y_scaled.values).view(-1, 1)
    
    X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)
    
    model = PlayerRatingPredictor(X.shape[1])
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    losses = []
    print("🧠 Training PyTorch Model...")
    for epoch in range(1000):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train) / 100  
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        
        if epoch % 200 == 0:
            print(f'  Epoch {epoch}: Loss = {loss.item():.4f}')
    
    model.eval()
    with torch.no_grad():
        train_pred = model(X_train)
        test_pred = model(X_test)
        
        train_r2 = r2_score(y_train.numpy().flatten() * 100, train_pred.numpy().flatten())
        test_r2 = r2_score(y_test.numpy().flatten() * 100, test_pred.numpy().flatten())
    
    print(f"PyTorch NN: Train R² = {train_r2:.3f}, Test R² = {test_r2:.3f}")
    return model, losses

pytorch_model, losses = train_pytorch_model(X_scaled, y_target)

🧠 Training PyTorch Model...
  Epoch 0: Loss = 0.0541
  Epoch 200: Loss = 0.0006
  Epoch 400: Loss = 0.0003
  Epoch 600: Loss = 0.0002
  Epoch 800: Loss = 0.0002
PyTorch NN: Train R² = 0.905, Test R² = 0.852


In [14]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_scaled, y_target)
rf_predictions = rf_model.predict(X_scaled)
rf_r2 = r2_score(y_target, rf_predictions)
rf_mae = mean_absolute_error(y_target, rf_predictions)

print(f"Random Forest: R² = {rf_r2:.3f}, MAE = {rf_mae:.2f}")

Random Forest: R² = 0.998, MAE = 0.03


In [15]:
def make_predictions(model, player_data, feature_names, scaler, model_type='pytorch'):
    X_new = player_data[feature_names].copy().fillna(player_data[feature_names].median())
    X_scaled = scaler.transform(X_new)
    
    if model_type == 'pytorch':
        model.eval()
        X_tensor = torch.FloatTensor(X_scaled)
        
        with torch.no_grad():
            predictions = model(X_tensor).numpy().flatten()
            predictions = np.clip(predictions, 0, 100)
    else:  
        predictions = model.predict(X_scaled)
        predictions = np.clip(predictions, 0, 100)
    
    return predictions

pytorch_predictions = make_predictions(pytorch_model, merged_data, feature_names, feature_scaler, 'pytorch')
rf_predictions = make_predictions(rf_model, merged_data, feature_names, feature_scaler, 'rf')

results_df = merged_data[['Player', 'Nation', 'Pos', 'Squad', 'Age', 'FIFA_Overall', 'Cluster']].copy()
results_df['Predicted_Rating_PyTorch'] = pytorch_predictions
results_df['Predicted_Rating_RF'] = rf_predictions
results_df['Difference_PyTorch'] = results_df['Predicted_Rating_PyTorch'] - results_df['FIFA_Overall']
results_df['Difference_RF'] = results_df['Predicted_Rating_RF'] - results_df['FIFA_Overall']

In [16]:
print(f"\n🏆 TOP 20 SPIELER NACH VORHERSAGE:")
top_players = results_df.nlargest(20, 'Predicted_Rating_PyTorch')[
    ['Player', 'Pos', 'Squad', 'FIFA_Overall', 'Predicted_Rating_PyTorch', 'Difference_PyTorch']
]
print(top_players.round(1))

print(f"\n🔍 INTERESSANTE FÄLLE (Große Unterschiede):")
underrated = results_df.nlargest(80, 'Difference_PyTorch')[
    ['Player', 'Pos', 'Squad', 'FIFA_Overall', 'Predicted_Rating_PyTorch', 'Difference_PyTorch']
]
print("Underrated (Model > FIFA):")
print(underrated.round(1))
 
overrated = results_df.nsmallest(10, 'Difference_PyTorch')[
    ['Player', 'Pos', 'Squad', 'FIFA_Overall', 'Predicted_Rating_PyTorch', 'Difference_PyTorch']
]
print("\nOverrated (FIFA > Model):")
print(overrated.round(1))


🏆 TOP 20 SPIELER NACH VORHERSAGE:
                  Player   Pos            Squad  FIFA_Overall  \
421       Joshua Kimmich    MF    Bayern Munich            89   
65       Kostas Tsimikas    DF        Liverpool            77   
659         Ben Chilwell    DF          Chelsea            82   
237  Benjamin Bourigeaud  FWMF           Rennes            81   
594      Jonathan Clauss    DF             Lens            80   
499       Kerem Demirbay    MF       Leverkusen            81   
786        Dimitri Payet  FWMF        Marseille            82   
850     Federico Dimarco    DF            Inter            78   
892      Kevin De Bruyne    MF  Manchester City            91   
504         Riyad Mahrez    FW  Manchester City            86   
864        Daniel Parejo    MF       Villarreal            86   
884        Adnan Januzaj  FWMF    Real Sociedad            81   
570          Nabil Fekir    MF            Betis            85   
988         Manuel Neuer    GK    Bayern Munich        

In [17]:
results_df[
    ['Player', 'Pos', 'Squad', 'FIFA_Overall', 'Predicted_Rating_PyTorch', 'Difference_PyTorch']
]

,Player,Pos,Squad,FIFA_Overall,Predicted_Rating_PyTorch,Difference_PyTorch
0,Jacob Barrett Laursen,DF,Arminia,69,73.152138,4.152138
1,Cristian Ansaldi,DF,Torino,76,77.636833,1.636833
2,Ludovic Ajorque,FW,Strasbourg,79,77.285423,-1.714577
3,Gabriel Paulista,DF,Valencia,82,80.049385,-1.950615
4,Ollie Watkins,FW,Aston Villa,78,76.051872,-1.948128
...,...,...,...,...,...,...
1000,Moussa Djenepo,MF,Southampton,74,72.120544,-1.879456
1001,Martin Adeline,MFFW,Reims,65,73.341255,8.341255
1002,Azor Matusiwa,MF,Reims,74,72.312813,-1.687187
1003,Iddrisu Baba,MF,Mallorca,75,73.242790,-1.757210


In [ ]:
print(f"\nDETAILLIERTE CLUSTER-ANALYSE:")
cluster_details = results_df.groupby('Cluster').agg({
    'Player': 'count',
    'FIFA_Overall': ['min', 'max', 'mean'],
    'Predicted_Rating_PyTorch': ['min', 'max', 'mean'],
    'Age': 'mean',
    'Pos': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Mixed'
}).round(1)

cluster_details.columns = ['Anzahl', 'FIFA_Min', 'FIFA_Max', 'FIFA_Mean', 
                          'Pred_Min', 'Pred_Max', 'Pred_Mean', 'Age_Mean', 'Haupt_Position']
print(cluster_details)

print(f"\n👥 BEISPIEL-SPIELER PRO CLUSTER:")
for cluster in sorted(results_df['Cluster'].unique()):
    cluster_players = results_df[results_df['Cluster'] == cluster]
    top_player = cluster_players.nlargest(1, 'Predicted_Rating_PyTorch').iloc[0]
    avg_player = cluster_players.nlargest(5, 'Predicted_Rating_PyTorch').iloc[2]  
    
    print(f"\nCluster {cluster}:")
    print(f"  Top:    {top_player['Player']} ({top_player['Pos']}) - {top_player['Predicted_Rating_PyTorch']:.1f}")
    print(f"  Avg:    {avg_player['Player']} ({avg_player['Pos']}) - {avg_player['Predicted_Rating_PyTorch']:.1f}")
    print(f"  Größe:  {len(cluster_players)} Spieler")


📈 DETAILLIERTE CLUSTER-ANALYSE:
         Anzahl  FIFA_Min  FIFA_Max  FIFA_Mean   Pred_Min   Pred_Max  \
Cluster                                                                
0           158        53        89       77.6  63.799999  98.199997   
1            71        65        90       78.2  68.900002  89.099998   
2           268        52        84       72.8  51.599998  92.800003   
3           202        50        89       73.5  50.700001  86.099998   
4           191        47        90       74.6  51.000000  86.000000   
5           174        55        91       77.0  68.000000  97.500000   

         Pred_Mean  Age_Mean Haupt_Position  
Cluster                                      
0        79.699997      26.6             MF  
1        80.699997      29.1             GK  
2        74.500000      25.5             DF  
3        73.300003      25.6             FW  
4        73.400002      26.7             DF  
5        81.900002      26.0           FWMF  

👥 BEISPIEL-SPIELER PR


Cluster 2:
  Top:    Ashley Westwood (MF) - 92.8
  Avg:    Marc Albrighton (FWDF) - 90.4
  Größe:  268 Spieler

Cluster 3:
  Top:    Harry Kane (FW) - 86.1
  Avg:    Erling Haaland (FW) - 85.4
  Größe:  202 Spieler

Cluster 4:
  Top:    Virgil van Dijk (DF) - 86.0
  Avg:    Presnel Kimpembe (DF) - 82.5
  Größe:  191 Spieler

Cluster 5:
  Top:    Benjamin Bourigeaud (FWMF) - 97.5
  Avg:    Dimitri Payet (FWMF) - 96.8
  Größe:  174 Spieler
